In [8]:
import requests
import pandas as pd
import re
import time
import numpy as np
from bs4 import BeautifulSoup
import json
import io, time, json

import csv
import gzip
import math
import hashlib
import requests

import itertools

In [9]:
namesDf = pd.read_csv("https://raw.githubusercontent.com/suzannenie/36490coldwar/main/names.csv")
# print(namesDf)

s = set()
for index, row in namesDf.iterrows():
  # print(row['firstname'], row['lastname'])
  firstname, lastname = row['firstname'], row['lastname']
  if type(firstname) == float:
    s.add(lastname)
  elif type(lastname) == float:
    s.add(firstname)
  else:
    s.add(firstname + " " + lastname)

namePairs = list(itertools.combinations(s, 2))

In [10]:
def make_adjacency_matrix(M, names):
    n = len(names)
    print(M.shape, n)
    A = np.zeros(n * n)
    A = A.reshape((n,n))
    print(A.shape)
    for doc in M:
        for i in range(n):
            if doc[i] != 0:
                for j in range(i+1, len(doc)):
                    if doc[j] != 0:
                        A[i][j] += 1
                        A[j][i] += 1
    print(A)
    return A


In [11]:
def get_pairs(A, df):
    pairs = []
    for i in range(0, len(df.columns.values)-1):
        for j in range(0, len(df.columns.values)-1): 
            if A[i][j] != 0:
                rowname = df.columns.values[i]
                colname = df.columns.values[j]
                pair = list((rowname, colname))
                pairs.append(pair)
    return pairs

In [12]:
from numpy import genfromtxt
filename = 'frequency.csv'
df = pd.read_csv(filename)
mydata = genfromtxt(filename, delimiter=',')
mydata = mydata[1:,1:]
names = list(df.columns)[1:]
txtfiles = list(df["id"])
A = make_adjacency_matrix(mydata, names)
namePairs = get_pairs(A, df)

(111, 108) 108
(108, 108)
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [13]:
namePairs

[['id', 'barbour'],
 ['id', 'carter'],
 ['id', 'drumond'],
 ['id', 'durand'],
 ['id', 'ermenji'],
 ['id', 'hoover'],
 ['id', 'kici'],
 ['id', 'kupi'],
 ['id', 'leiser'],
 ['id', 'mikolajczk'],
 ['id', 'nagy'],
 ['id', 'pali'],
 ['id', 'rearns'],
 ['id', 'repishti'],
 ['id', 'sheltt'],
 ['id', 'spahiu'],
 ['id', 'wailes'],
 ['adenauer', 'drumond'],
 ['aschan', 'barnes'],
 ['aschan', 'drumond'],
 ['aschan', 'koch'],
 ['aschan', 'stein'],
 ['aschan', 'willi'],
 ['barbour', 'id'],
 ['barbour', 'bruce'],
 ['barbour', 'ermenji'],
 ['barbour', 'kelly'],
 ['barbour', 'pali'],
 ['barbour', 'raue'],
 ['barbour', 'rhee'],
 ['barbour', 'tenef'],
 ['barbour', 'wailes'],
 ['barbour', 'willi'],
 ['barnes', 'aschan'],
 ['barnes', 'drumond'],
 ['barnes', 'gland'],
 ['barnes', 'graf'],
 ['barnes', 'koch'],
 ['barnes', 'mathews'],
 ['barnes', 'stevens'],
 ['barnes', 'willi'],
 ['bennet', 'drumond'],
 ['bennet', 'göttling'],
 ['bennet', 'jackson'],
 ['bennet', 'juhast'],
 ['bennet', 'lawrence'],
 ['bennet

In [14]:
 query = "https://www.googleapis.com/books/v1/volumes?q=kennan+wisner"
response = requests.get(query).json()
print(response.get('items'))

[{'kind': 'books#volume', 'id': 'iy7aCwAAQBAJ', 'etag': 'mflj11u+ZBA', 'selfLink': 'https://www.googleapis.com/books/v1/volumes/iy7aCwAAQBAJ', 'volumeInfo': {'title': 'The Georgetown Set', 'subtitle': 'Friends and Rivals in Cold War Washington', 'authors': ['Gregg Herken'], 'publisher': 'Vintage', 'publishedDate': '2015-11-24', 'description': 'Originally published: New York: Alfred A. Knopf, 2014.', 'industryIdentifiers': [{'type': 'ISBN_13', 'identifier': '9780307456342'}, {'type': 'ISBN_10', 'identifier': '030745634X'}], 'readingModes': {'text': False, 'image': False}, 'pageCount': 528, 'printType': 'BOOK', 'categories': ['History'], 'averageRating': 5, 'ratingsCount': 1, 'maturityRating': 'NOT_MATURE', 'allowAnonLogging': False, 'contentVersion': '0.3.0.0.preview.0', 'panelizationSummary': {'containsEpubBubbles': False, 'containsImageBubbles': False}, 'imageLinks': {'smallThumbnail': 'http://books.google.com/books/content?id=iy7aCwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&so

In [15]:
#from typing_extensions import TypeVarTuple
links = []
for (person1, person2) in namePairs:
  # get just lastname (or firstname if not available)
  q1, q2 = person1[-1], person2[-1]
  query = "https://www.googleapis.com/books/v1/volumes?q=" + q1 + "+" + q2

  # list of python dictionaries as elements, each
  # corresponding to json data for a given book
  # returned from the search query
  response = requests.get(query).json()
  #print(response)
  if response.get('items')!= None:
      for k in response.get('items'):
        #print(person1, person2)
        #print(k['volumeInfo']['previewLink'].find("PA"))
        link = k['volumeInfo']['previewLink']
        
        if "PA" in link: 
          num = re.search("(?:PA)\d+", link)
          if num != None:
            num = int(re.findall(r'\d+', num.group(0))[0])
            links.append([(person1, person2), link, num])
print(links)

[[('id', 'drumond'), 'http://books.google.com/books?id=lo6OFEH1ezEC&pg=PA1&dq=d+d&hl=&cd=6&source=gbs_api', 1], [('id', 'drumond'), 'http://books.google.com/books?id=5SNfEAAAQBAJ&pg=PA376&dq=d+d&hl=&cd=7&source=gbs_api', 376], [('id', 'drumond'), 'http://books.google.com/books?id=kohJAAAAYAAJ&pg=PA65&dq=d+d&hl=&cd=8&source=gbs_api', 65], [('id', 'drumond'), 'http://books.google.com/books?id=zQ7ez-8CWkgC&pg=PA14&dq=d+d&hl=&cd=9&source=gbs_api', 14], [('id', 'durand'), 'http://books.google.com/books?id=lo6OFEH1ezEC&pg=PA1&dq=d+d&hl=&cd=6&source=gbs_api', 1], [('id', 'durand'), 'http://books.google.com/books?id=5SNfEAAAQBAJ&pg=PA376&dq=d+d&hl=&cd=7&source=gbs_api', 376], [('id', 'durand'), 'http://books.google.com/books?id=kohJAAAAYAAJ&pg=PA65&dq=d+d&hl=&cd=8&source=gbs_api', 65], [('id', 'durand'), 'http://books.google.com/books?id=zQ7ez-8CWkgC&pg=PA14&dq=d+d&hl=&cd=9&source=gbs_api', 14], [('id', 'ermenji'), 'http://books.google.com/books?id=ZyjGqJfUtm0C&pg=PA18&dq=d+i&hl=&cd=1&source=g

In [16]:
with open("links.txt", "w") as output:
    for row in links:
        output.write(str(row) + '\n')

In [17]:
len(links)

2475

In [18]:
import os
import urllib
import traceback
import regex as re
from time import sleep
from seleniumwire import webdriver
from progressbar import progressbar as bar
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

import ssl
ssl._create_default_https_context = ssl._create_unverified_context


from webdriver_manager.chrome import ChromeDriverManager


In [19]:
def get_book_url(url):
    if re.findall(r"id=[A-Za-z0-9]+", url):
        id_part = re.findall(r"id=[A-Za-z0-9]+", url)[-1]
    else:
        print("Invalid input. Please try again.")
        get_book_url()
        
    return (f"https://books.google.com/books?{id_part}&pg=1&hl=en#v=onepage&q&f=false",
            f"https://books.google.com/books?{id_part}&pg=1&hl=en&f=false&output=embed&source=gbs_embed")

def get_book_data(url):
    """
    Inspects the opened book, returns a
    `str` with its title and author.
    """
    driver.get(url)
    driver.refresh()
    sleep(3)
    title = driver.find_element_by_class_name("gb-volume-title").text
    author = driver.find_element_by_class_name("addmd").text    
    return f"{title} (b{author[1:]})"

def capture_requests(url):
    """
    Scrolls through the whole book,
    returns the requests driver made.
    """
    driver.get(url)
    driver.refresh()
    sleep(5)
    checkpoint = ""
    
    while checkpoint != driver.find_element_by_class_name("pageImageDisplay"):
        checkpoint = driver.find_element_by_class_name("pageImageDisplay")
        checkpoint.click()
        # scrolling ~25 pages
        for i in range(25):
            html = driver.find_element_by_tag_name("body")
            html.click()
            html.send_keys(Keys.SPACE)
        sleep(2)
    return str(driver.requests)

def extract_urls(requests):
    """
    Takes driver's requests as an input,
    returns a `dict` of page image URLs.
    """
    urls = re.findall(r"url='(https:\/\/[^']+content[^']+pg=[A-Z]+([0-9]+)[^']+)(&w=[0-9]+)'", requests)

    return {int(url[1]): url[0] + "&w=69420" for url in urls}



def select_pages(user_input, all_pages):
    """
    Takes the range of pages user specified
    and image URLs of all pages available,
    returns a `dict` with selected pages only.
    """
    ranges = user_input.replace(" ", "").split(",")
    page_numbers = []
    
    if "all" in ranges:
        return all_pages
    while "odd" in ranges:
        page_numbers.extend([i for i in all_pages.items() if i[0] % 2])
        ranges.remove("odd")
    while "even" in ranges:
        page_numbers.extend([i for i in all_pages.items() if i[0] % 2 == 0])
        ranges.remove("even")
    for segment in ranges:
        if "-" in segment:
            a, b = segment.split("-")
            page_numbers.extend([i for i in all_pages.items() if int(a) <= i[0] <= int(b)])
        elif int(segment) in all_pages.keys():
            page_numbers.append((int(segment), all_pages[int(segment)]))

    return dict(set(page_numbers))
    
def get_cookie(url):
    """
    Driver needs to behave like a real
    user to GET page images. This function
    returns a cookie to bribe Google with.
    """
    cookies = []
    driver.get(url)
    driver.refresh()
    
    for request in driver.requests:
        if request.headers:
           for k, v in request.headers.items():
                if k.lower() == "cookie":
                    cookies.append(v)
                    break

    return cookies[0]

def download_imgs(pages, cookie, directory):
    """
    Takes the `dict` of pages to download,
    the cookie to use and the directory
    to save to, and then does the magic.
    """
    proxy = urllib.request.ProxyHandler({})
    opener = urllib.request.build_opener(proxy)
    opener.addheaders = [("user-agent", "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_4) AppleWebKit/603.1.30 (KHTML, like Gecko) Version/10.1 Safari/603.1.30"),
                        ("cookie", cookie)]
    urllib.request.install_opener(opener)

    print(pages.items())
    for number, url in pages.items():
        urllib.request.urlretrieve(url, os.path.join(directory, f"page{number}.png"))



In [20]:
def step1(url):
    global book_data, all_pages
    
    from_url = url

    data_url, pages_url = get_book_url(url)
    book_data = get_book_data(data_url)
    print(f"\nWe will now process the pages of \"{book_data}\" one by one. Sit back and relax, as this may take some time, depending on the number of its pages.\n")
    reqs = capture_requests(pages_url)
    all_pages = extract_urls(reqs)
    print("""Now that most of the job is done (yahoo!), it is highly recommended to backup the current progress we have made, so as not to lose it if an error happens to be thrown afterward.
Also, if you would like to download another segment of this book later, the backup will be used then to save your precious time.""")


In [21]:
def step2(selection):
    global selected_pages, cookie
    
    try:
        selected_pages = select_pages(selection, all_pages)
        
    except:
        print("Invalid input. Please try again.")
        step2()

    # it's a surprise tool that will help us later
    cookie = get_cookie(list(all_pages.items())[0][1])

In [22]:
def step3():    
    current_directory = os.getcwd()
    main_directory = os.path.join(current_directory, r'books')
    new_directory = os.path.join(main_directory, book_data)
    if not os.path.exists(new_directory):
        os.makedirs(new_directory)

        
    print(f"\nWe will now download all {len(selected_pages)} pages you selected. This will take a minute or two.\n")
    download_imgs(selected_pages, cookie, new_directory)

In [25]:
if __name__ == "__main__":
    global driver
    
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--log-level=3")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
    chrome_options.add_experimental_option("prefs", {"safebrowsing.enabled": True})
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

    #url = "http://books.google.com/books?id=DTtAoKLXuz8C&pg=PA117&dq=a+u&hl=&cd=1&source=gbs_api"
    #selection = "1"
    #step1(url)
    #step2(selection)
    #step3()

    for link in links:
        try:
            url = link[-2]
            selection = str(link[-1])
            print(url)
            print(selection)
            step1(url)
            print("step1")
            step2(selection)
            print("step2")
            step3()
            time.sleep(1)
        except:
            print("failed", link)
            time.sleep(5)
   



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/yingxuehu/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


http://books.google.com/books?id=lo6OFEH1ezEC&pg=PA1&dq=d+d&hl=&cd=6&source=gbs_api
1

We will now process the pages of "MAC Transportation Authorization (MTA), DD Forms 1482-1 and 4: Military Airlift (by United States. Department of the Air Force)" one by one. Sit back and relax, as this may take some time, depending on the number of its pages.

Now that most of the job is done (yahoo!), it is highly recommended to backup the current progress we have made, so as not to lose it if an error happens to be thrown afterward.
Also, if you would like to download another segment of this book later, the backup will be used then to save your precious time.
step1
step2

We will now download all 1 pages you selected. This will take a minute or two.

dict_items([(1, 'https://books.google.com/books/content?id=lo6OFEH1ezEC&pg=PA1&img=1&zoom=3&hl=en&sig=ACfU3U3I_y7Qg6zfoRZfjhduoKLzwyaMEg&w=69420')])
http://books.google.com/books?id=5SNfEAAAQBAJ&pg=PA376&dq=d+d&hl=&cd=7&source=gbs_api
376

We will now

Now that most of the job is done (yahoo!), it is highly recommended to backup the current progress we have made, so as not to lose it if an error happens to be thrown afterward.
Also, if you would like to download another segment of this book later, the backup will be used then to save your precious time.
step1
step2

We will now download all 1 pages you selected. This will take a minute or two.

dict_items([(71, 'https://books.google.com/books/content?id=SCE9AAAAYAAJ&pg=PA71&img=1&zoom=3&hl=en&sig=ACfU3U0TmKoPmfxV5mNgF0mADfwSbZzUuQ&w=69420')])
http://books.google.com/books?id=jNcNAAAAQAAJ&pg=PA215&dq=d+i&hl=&cd=7&source=gbs_api
215

We will now process the pages of "Saggio di agricoltura per le isole di Malta, e Gozo (by Carlo Giacinto (Carmelitano scalzo.))" one by one. Sit back and relax, as this may take some time, depending on the number of its pages.

Now that most of the job is done (yahoo!), it is highly recommended to backup the current progress we have made, so as not to lose

http://books.google.com/books?id=Yy1GEAAAQBAJ&pg=PA158&dq=d+i&hl=&cd=8&source=gbs_api
158

We will now process the pages of "Il Duomo di Siena: Excavations and Pottery below the Siena Cathedral (by Gabriele Castiglia)" one by one. Sit back and relax, as this may take some time, depending on the number of its pages.

Now that most of the job is done (yahoo!), it is highly recommended to backup the current progress we have made, so as not to lose it if an error happens to be thrown afterward.
Also, if you would like to download another segment of this book later, the backup will be used then to save your precious time.
step1
step2

We will now download all 1 pages you selected. This will take a minute or two.

dict_items([(158, 'https://books.google.com/books/content?id=jNcNAAAAQAAJ&pg=PA158&img=1&zoom=3&hl=en&sig=ACfU3U0u7YJuvflvt0Vm62Y7hBEOhXPhUw&w=69420')])
http://books.google.com/books?id=3VxaBgAAQBAJ&pg=PA45&dq=d+i&hl=&cd=9&source=gbs_api
45

We will now process the pages of " (b)" 

step2

We will now download all 1 pages you selected. This will take a minute or two.

dict_items([(45, 'https://books.google.com/books/content?id=jNcNAAAAQAAJ&pg=PA45&img=1&zoom=3&hl=en&sig=ACfU3U20hTRKUTsyfH70Uy0mPJ6LVUWogw&w=69420')])
http://books.google.com/books?id=JuA7JVDoCfAC&pg=PA12-IA5&dq=d+i&hl=&cd=10&source=gbs_api
12

We will now process the pages of "Historia delle guerre ciuili di Francia ... Nella quale si contengono le ... (by Enrico Caterino DAVILA)" one by one. Sit back and relax, as this may take some time, depending on the number of its pages.

failed [('id', 'kupi'), 'http://books.google.com/books?id=JuA7JVDoCfAC&pg=PA12-IA5&dq=d+i&hl=&cd=10&source=gbs_api', 12]
http://books.google.com/books?id=ZFUzEAAAQBAJ&pg=PA1&dq=d+k&hl=&cd=1&source=gbs_api
1

We will now process the pages of "Allies: Real Talk About Showing Up, Screwing Up, And Trying Again (by Shakirah Bourne, Dana Alison Levy)" one by one. Sit back and relax, as this may take some time, depending on the numb

...

http://books.google.com/books?id=e9ahW8XAFXkC&pg=PA151&dq=f+e&hl=&cd=8&source=gbs_api
151
failed [('tenef', 'lawrence'), 'http://books.google.com/books?id=e9ahW8XAFXkC&pg=PA151&dq=f+e&hl=&cd=8&source=gbs_api', 151]
http://books.google.com/books?id=2ua9F01XZWYC&pg=PA137&dq=f+e&hl=&cd=9&source=gbs_api
137
failed [('tenef', 'lawrence'), 'http://books.google.com/books?id=2ua9F01XZWYC&pg=PA137&dq=f+e&hl=&cd=9&source=gbs_api', 137]
http://books.google.com/books?id=FLhmQ0qP-TAC&pg=PA50&dq=f+s&hl=&cd=3&source=gbs_api
50
failed [('tenef', 'phillips'), 'http://books.google.com/books?id=FLhmQ0qP-TAC&pg=PA50&dq=f+s&hl=&cd=3&source=gbs_api', 50]
http://books.google.com/books?id=9feE9VbJsB0C&pg=PA43&dq=f+s&hl=&cd=4&source=gbs_api
43
failed [('tenef', 'phillips'), 'http://books.google.com/books?id=9feE9VbJsB0C&pg=PA43&dq=f+s&hl=&cd=4&source=gbs_api', 43]
http://books.google.com/books?id=V_sxAQAAMAAJ&pg=PA34&dq=f+s&hl=&cd=5&source=gbs_api
34
failed [('tenef', 'phillips'), 'http://books.google.com/book

http://books.google.com/books?id=aE44YA-ovrIC&pg=PA42&dq=n+k&hl=&cd=2&source=gbs_api
42
failed [('thompson', 'kirkpatrick'), 'http://books.google.com/books?id=aE44YA-ovrIC&pg=PA42&dq=n+k&hl=&cd=2&source=gbs_api', 42]
http://books.google.com/books?id=EJT6DwAAQBAJ&pg=PA43&dq=n+k&hl=&cd=3&source=gbs_api
43
failed [('thompson', 'kirkpatrick'), 'http://books.google.com/books?id=EJT6DwAAQBAJ&pg=PA43&dq=n+k&hl=&cd=3&source=gbs_api', 43]
http://books.google.com/books?id=YxcIQSU0FPYC&pg=PA116&dq=n+k&hl=&cd=4&source=gbs_api
116
failed [('thompson', 'kirkpatrick'), 'http://books.google.com/books?id=YxcIQSU0FPYC&pg=PA116&dq=n+k&hl=&cd=4&source=gbs_api', 116]
http://books.google.com/books?id=YI1eEAAAQBAJ&pg=PA101&dq=n+k&hl=&cd=5&source=gbs_api
101
failed [('thompson', 'kirkpatrick'), 'http://books.google.com/books?id=YI1eEAAAQBAJ&pg=PA101&dq=n+k&hl=&cd=5&source=gbs_api', 101]
http://books.google.com/books?id=CLdADwAAQBAJ&pg=PA47&dq=n+k&hl=&cd=6&source=gbs_api
47
failed [('thompson', 'kirkpatrick'),

http://books.google.com/books?id=8LS2DwAAQBAJ&pg=PA3&dq=n+n&hl=&cd=6&source=gbs_api
3
failed [('thompson', 'stassen'), 'http://books.google.com/books?id=8LS2DwAAQBAJ&pg=PA3&dq=n+n&hl=&cd=6&source=gbs_api', 3]
http://books.google.com/books?id=gRmQppVe6dkC&pg=PA34&dq=n+n&hl=&cd=9&source=gbs_api
34
failed [('thompson', 'stassen'), 'http://books.google.com/books?id=gRmQppVe6dkC&pg=PA34&dq=n+n&hl=&cd=9&source=gbs_api', 34]
http://books.google.com/books?id=Yq42AQAAMAAJ&pg=RA2-PA4&dq=n+f&hl=&cd=1&source=gbs_api
4
failed [('thompson', 'tenef'), 'http://books.google.com/books?id=Yq42AQAAMAAJ&pg=RA2-PA4&dq=n+f&hl=&cd=1&source=gbs_api', 4]
http://books.google.com/books?id=fqI2AQAAMAAJ&pg=RA4-PA1&dq=n+f&hl=&cd=2&source=gbs_api
1
failed [('thompson', 'tenef'), 'http://books.google.com/books?id=fqI2AQAAMAAJ&pg=RA4-PA1&dq=n+f&hl=&cd=2&source=gbs_api', 1]
http://books.google.com/books?id=ZZ02AQAAMAAJ&pg=PA218&dq=n+f&hl=&cd=3&source=gbs_api
218
failed [('thompson', 'tenef'), 'http://books.google.com/bo

http://books.google.com/books?id=RKo1AQAAMAAJ&pg=RA1-PA8&dq=s+r&hl=&cd=10&source=gbs_api
8
failed [('wailes', 'leiser'), 'http://books.google.com/books?id=RKo1AQAAMAAJ&pg=RA1-PA8&dq=s+r&hl=&cd=10&source=gbs_api', 8]
http://books.google.com/books?id=m7nzdn2E1FkC&pg=PA20&dq=i+n&hl=&cd=7&source=gbs_api
20
failed [('willi', 'aschan'), 'http://books.google.com/books?id=m7nzdn2E1FkC&pg=PA20&dq=i+n&hl=&cd=7&source=gbs_api', 20]
http://books.google.com/books?id=3qXj04ccDfoC&pg=PA38&dq=i+r&hl=&cd=3&source=gbs_api
38
failed [('willi', 'barbour'), 'http://books.google.com/books?id=3qXj04ccDfoC&pg=PA38&dq=i+r&hl=&cd=3&source=gbs_api', 38]
http://books.google.com/books?id=k-lUAAAAcAAJ&pg=RA2-PA264&dq=i+r&hl=&cd=5&source=gbs_api
264
failed [('willi', 'barbour'), 'http://books.google.com/books?id=k-lUAAAAcAAJ&pg=RA2-PA264&dq=i+r&hl=&cd=5&source=gbs_api', 264]
http://books.google.com/books?id=MuZUAAAAcAAJ&pg=RA1-PA1370&dq=i+r&hl=&cd=6&source=gbs_api
1370
failed [('willi', 'barbour'), 'http://books.goo

http://books.google.com/books?id=m3mpAgAAQBAJ&pg=PA60&dq=i+r&hl=&cd=8&source=gbs_api
60
failed [('willi', 'meeker'), 'http://books.google.com/books?id=m3mpAgAAQBAJ&pg=PA60&dq=i+r&hl=&cd=8&source=gbs_api', 60]
http://books.google.com/books?id=us6FDwAAQBAJ&pg=PA35&dq=i+r&hl=&cd=9&source=gbs_api
35
failed [('willi', 'meeker'), 'http://books.google.com/books?id=us6FDwAAQBAJ&pg=PA35&dq=i+r&hl=&cd=9&source=gbs_api', 35]
http://books.google.com/books?id=m7nzdn2E1FkC&pg=PA20&dq=i+n&hl=&cd=7&source=gbs_api
20
failed [('willi', 'morgan'), 'http://books.google.com/books?id=m7nzdn2E1FkC&pg=PA20&dq=i+n&hl=&cd=7&source=gbs_api', 20]
http://books.google.com/books?id=JdpCXrUPPEQC&pg=PA9&dq=i+n&hl=&cd=10&source=gbs_api
9
failed [('willi', 'morgan'), 'http://books.google.com/books?id=JdpCXrUPPEQC&pg=PA9&dq=i+n&hl=&cd=10&source=gbs_api', 9]
http://books.google.com/books?id=3qXj04ccDfoC&pg=PA38&dq=i+r&hl=&cd=3&source=gbs_api
38
failed [('willi', 'morner'), 'http://books.google.com/books?id=3qXj04ccDfoC&pg

In [ ]:
step1("http://books.google.com/books?id=DTtAoKLXuz8C&pg=PA117&dq=a+u&hl=&cd=1&source=gbs_api")
step2("117")
step3()